# Genetic Algorithm Approach

## Setup

Import necessary libraries and load the dataset.

In [1]:
import networkx as nx
import random
import numpy as np
from itertools import permutations
from tqdm import tqdm
import copy

In [2]:
g = nx.read_gml("./graphs/seidel_rally_graph-12.gml")
g.nodes

NodeView(('Weinstadt', 'Klub Gru', 'Glashütte', 'Das Torberg', 'Mokador 8', 'u.s.w.', 'Pub by Charles', 'Hopfen-Stuben', 'Café Amadeus', 'Acabar', 'Ganz Wien', 'Café 7*stern', 'Kerstin', 'Prater Dome', 'Zum Schlauch', 'Oskar', 'Bastille-Pub', 'Gasthaus Manu', 'Schlupfwinkel', 'Wiener Blut', 'Cafe Nest', 'Derwisch', 'Wirr am Brunnenmarkt', 'Weberknecht', "Wind & Mill's", 'Miranda', 'Top Kino Bar', 'Rendezvous', 'Mimi im Stadtelefant', "Guggi's Beisl", 'Azul Bar', 'Casablanca', 'Slammerbar', 'Red Lion', "'s Stammbeisl", 'Bar Black Lounge', 'Café Frog', 'Gimlet', 'Remi Demi', 'Espresso Slatki San', 'Gerüchteküche', 'Espresso Karawane', "Karin's Radlertreff", 'SLU-Bar', 'Unders', 'Couch Potato', "Steiner's Club Lounge", 'K.St.V. Rhaetia', 'Schaumweinhäuschen', 'Schneider-Gössl Sektbar', 'Kleine Ober St. Veiter Bierstube', "Gaby's Treff", 'Biergartl', 'The Twinspub', 'Flokal', "Wistl's Cafe", "Otti's Würstel Bar", 'Hinteralm', 'Cafe Liquid', 'The Little Stage', 'Stammersdorfer Hauptprostamt

## Create initital Population

In [3]:
def enumerate_reversed(l):
    return zip(range(len(l)-1, -1, -1), reversed(l))

In [4]:
def init_pop(bar_graph, n_population):
    """
    Generate a randomly initialized population of bar routes
    Input:
        1. bar_graph: Graph of all bars
        2. n_population: Size of initial population
    Output:
        List of initial population
    """

    population = []

    for _ in range(n_population):
        route = []

        for idx in range(23, 0, -1):
            district_bars = [node for node, value in bar_graph.nodes(data=True) if value["district"] == idx]
            selected_bar = np.random.choice(district_bars)
            route.append(selected_bar)

        population.append(route)
    
    return population

## Create Fitness Function

In [5]:
def total_distance(graph, route):
    """
    Calculate total distance for a route.
    Input:
        graph: Graph of bars
        route: List of visited bars
    Output:
        tot_dist: The total distance of the route
    """

    tot_dist = 0
    for i in range(22):
        tot_dist += graph.get_edge_data(route[i], route[i+1])["weight"]
    
    return tot_dist

In [6]:
def fitness(graph, pop):
    """
    Calculate fitness for each route.
    Input: 
        graph: Graph of bars
        pop: List of routes
    Output:
        pop_fit: List of fitness
    """
    
    pop_fit = []
    for route in pop:
        tot_dist = total_distance(graph, route)
        pop_fit.append(1 / tot_dist)

    return pop_fit

## Create Selection Process

In [7]:
def roulette_wheel(pop_fitness, size_perc):
    """
    Create a roulette wheel that lets one random route survive.
    Probability is based on fitness.
    Input:
        pop_fitness: List of fitness
        size_perc: Percentage of population to select
    Output:
        idx: List of indexes of surviving units
    """

    population_fitness = np.array(pop_fitness)
    total_fitness = np.sum(pop_fitness)
    surv_prob = population_fitness / total_fitness

    cumsum_surv_prob = np.cumsum(surv_prob)

    idx = []
    for i in range(int(len(population_fitness) * size_perc)):
        rand_val = np.random.uniform(0, 1)
        selected_idx = np.where(cumsum_surv_prob >= rand_val)[0][0]
        idx.append(selected_idx)

    return idx

## Create Mutation Process

In [8]:
def mutation(graph, population, prob):
    """
    Mutate a random bar in a random district in each route by a certain probability.
    Input:
        graph: The graph containing the data
        population: List containing the routes
        prob: The porpability by which a mutation should happen
    Output:
        mutated_population: List containing the mutated routes
    """
    
    mutated_population = copy.deepcopy(population)
    mutation_mask = prob > np.random.uniform(0, 1, len(population))
    mutation_idx = np.argwhere(mutation_mask).flatten()
    dist_idx = list(range(23, 0, -1))

    for idx in mutation_idx:
        dist = np.random.randint(1, 24)
        dist_i = dist_idx[dist-1]
        
        current_bar = mutated_population[idx][dist_i-1]
        dist_bars = [node for node, value in graph.nodes(data=True) if value["district"] == dist]

        mutated_population[idx][dist_i-1] = np.random.choice([loc for loc in dist_bars if loc != current_bar])

    return mutated_population
        

## Create Crossover Process

In [9]:
def crossover(route_1, route_2):
    """
    Swap two routes after cutoff
    Input:
        route_1: List of bars
        route_2: Second list of bars
    Output:
        cross_1: New route
        cross_2: Second new route
    """
    
    cross_1 = []
    cross_2 = []
    for loc_1, loc_2 in zip(route_1, route_2):
        if np.random.rand() > 0.5:
            cross_1.append(loc_1)
            cross_2.append(loc_2)
        else:
            cross_1.append(loc_2)
            cross_2.append(loc_1)

    return cross_1, cross_2

## Putting it all together

In [10]:
def seidel_ralley_ga(bar_graph, epochs, n_pop, crossover_perc, mutation_prob):
    #create initial population
    population = init_pop(bar_graph, n_pop)

    for epoch in tqdm(range(epochs)):
        #get fitness of current population
        pop_fitness = fitness(bar_graph, population)

        #run selection process
        parents_idx = roulette_wheel(pop_fitness, crossover_perc)

        #run crossover
        children = []
        num_children = int(n_pop * crossover_perc)
        if num_children % 2 == 1:
            num_children -= 1
        for i in range(0, num_children, 2):
            child1, child2 = crossover(population[parents_idx[i]], population[parents_idx[i+1]])
            children.extend([child1, child2])
        
        #run mutation
        mutated_children = mutation(bar_graph, children, 0.05)

        #replace part of the population
        pop_fit_sort_idx = np.argsort(pop_fitness)[::-1]
        elite = [population[j] for j in pop_fit_sort_idx[:n_pop - len(mutated_children)]]

        mutated_children.extend(elite)
        population = mutated_children
        
    return population

In [ ]:
best_gen = seidel_ralley_ga(g, 50, 200, 0.8, 0.05)
pop_fitness = fitness(g, best_gen)

100%|██████████| 50/50 [00:00<00:00, 142.05it/s]


In [12]:
best_solution_idx = np.argsort(pop_fitness)[::-1]
best_solution = best_gen[best_solution_idx[0]]
distance = total_distance(g, best_solution)
distance

23061.0

In [13]:
for bar in best_solution:
    print(bar, [value["district"] for node, value in g.nodes(data=True) if node == bar])

Köö Billardcafe [23.0]
Otti's Würstel Bar [22.0]
Star Voice Lounge [21.0]
Couch Potato [20.0]
SLU-Bar [19.0]
Gasthaus Manu [18.0]
Espresso Karawane [17.0]
Derwisch [16.0]
Pub by Charles [15.0]
Gaby's Treff [14.0]
Schaumweinhäuschen [13.0]
Golden Harp Kaffeehaus [12.0]
Café Frog [11.0]
Mimi im Stadtelefant [10.0]
Klub Gru [9.0]
Das Torberg [8.0]
Café 7*stern [7.0]
Miranda [6.0]
Cafe Liquid [5.0]
Cafe Nest [4.0]
's Stammbeisl [3.0]
Kerstin [2.0]
Casablanca [1.0]
